In [35]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [36]:
def get_device():
    """
    Returns the device to be used for tensor computations (GPU if available, else CPU).
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')
    return device

device = get_device()

Using device: cpu


In [5]:
def load_dataset(file_path):
    """
    Loads the dataset from the specified CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: Loaded dataset as a pandas DataFrame.
    """
    df = pd.read_csv(file_path)
    return df

df = load_dataset('D:/Projects/EffectiveDayAI/schedules-ai-db/Daily_Records_Dataset.csv')

NameError: name 'pd' is not defined

In [38]:
def time_to_minutes(time_str):
    """
    Converts a time string in 'HH:MM' format to minutes from midnight.

    Args:
        time_str (str): Time string in 'HH:MM' format.

    Returns:
        int: Minutes from midnight.
    """
    h, m = map(int, time_str.split(':'))
    return h * 60 + m

def preprocess_data(df):
    """
    Preprocesses the dataset by converting time columns, encoding categorical variables,
    and normalizing numerical features.

    Args:
        df (pd.DataFrame): The raw dataset.

    Returns:
        pd.DataFrame: The preprocessed dataset.
    """
    df['Start_Minutes'] = df['Start_Time'].apply(time_to_minutes)
    df['End_Minutes'] = df['End_Time'].apply(time_to_minutes)

    df['Task_Duration'] = df['End_Minutes'] - df['Start_Minutes']

    le_task_type = LabelEncoder()
    df['Task_Type_Encoded'] = le_task_type.fit_transform(df['Task_Type'])

    le_preferred_time = LabelEncoder()
    df['Preferred_Time_Encoded'] = le_preferred_time.fit_transform(df['Preferred_Time'])

    df['Fixed_Task'] = df['Fixed_Task'].astype(int)

    numeric_features = ['Task_Priority', 'Task_Duration',
                        'Breaks_Count', 'Breaks_Duration', 'Sleep_Duration', 'Sleep_Quality',
                        'Energy_Level', 'Heart_Rate', 'Start_Minutes', 'End_Minutes']

    scaler = StandardScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])

    return df

df = preprocess_data(df)

In [39]:
def get_feature_columns():
    """
    Defines the list of feature columns to be used in the model.

    Returns:
        list: List of feature column names.
    """
    feature_columns = ['Task_Type_Encoded', 'Task_Priority', 'Task_Duration', 'Fixed_Task',
                       'Preferred_Time_Encoded', 'Sleep_Duration', 'Sleep_Quality',
                       'Energy_Level', 'Heart_Rate', 'Start_Minutes', 'End_Minutes']
    return feature_columns

feature_columns = get_feature_columns()

In [40]:
def prepare_sequences(df, feature_columns):
    """
    Groups tasks by date to create sequences and prepares per-task and overall targets.

    Args:
        df (pd.DataFrame): The preprocessed dataset.
        feature_columns (list): List of feature column names.

    Returns:
        sequences (list): List of task sequences.
        per_task_targets (list): List of per-task target arrays.
        overall_targets (list): List of overall target arrays.
    """
    grouped = df.groupby('Date')

    sequences = []
    per_task_targets = []
    overall_targets = []

    for date, group in grouped:
        group = group.sort_values('Start_Minutes')
        overall_efficiency = group['Overall_Efficiency_Score'].dropna().unique()
        overall_satisfaction = group['Overall_Satisfaction_Score'].dropna().unique()

        if len(overall_efficiency) > 0 and len(overall_satisfaction) > 0:
            overall_target = [overall_efficiency[0], overall_satisfaction[0]]
            overall_targets.append(overall_target)

            seq = group[feature_columns].values
            sequences.append(seq)

            efficiency_scores = group['Efficiency_Score'].values
            satisfaction_scores = group['Satisfaction_Score'].values
            per_task_target = np.column_stack((efficiency_scores, satisfaction_scores))
            per_task_targets.append(per_task_target)
        else:
            continue

    return sequences, per_task_targets, overall_targets

sequences, per_task_targets, overall_targets = prepare_sequences(df, feature_columns)

In [41]:
class TaskSequenceDataset(Dataset):
    """
    Custom Dataset class for task sequences and targets.

    Args:
        sequences (list): List of task sequences.
        per_task_targets (list): List of per-task target arrays.
        overall_targets (list): List of overall target arrays.
    """
    def __init__(self, sequences, per_task_targets, overall_targets):
        self.sequences = sequences
        self.per_task_targets = per_task_targets
        self.overall_targets = overall_targets

    def __len__(self):
        """
        Returns the number of sequences in the dataset.
        """
        return len(self.sequences)

    def __getitem__(self, idx):
        """
        Retrieves the sequence and targets at the specified index.

        Args:
            idx (int): Index of the data point.

        Returns:
            dict: Dictionary containing the sequence and targets.
        """
        seq = torch.tensor(self.sequences[idx], dtype=torch.float32)
        per_task_target = torch.tensor(self.per_task_targets[idx], dtype=torch.float32)
        overall_target = torch.tensor(self.overall_targets[idx], dtype=torch.float32)
        return {'sequence': seq, 'per_task_target': per_task_target, 'overall_target': overall_target}

In [42]:
def collate_fn(batch):
    """
    Collate function to handle variable-length sequences and pad them.

    Args:
        batch (list): List of data points.

    Returns:
        dict: Dictionary containing padded sequences and targets.
    """
    sequences = [item['sequence'] for item in batch]
    per_task_targets = [item['per_task_target'] for item in batch]
    overall_targets = torch.stack([item['overall_target'] for item in batch])

    sequences_padded = nn.utils.rnn.pad_sequence(sequences, batch_first=True)
    per_task_targets_padded = nn.utils.rnn.pad_sequence(per_task_targets, batch_first=True)

    return {
        'sequence': sequences_padded,
        'per_task_target': per_task_targets_padded,
        'overall_target': overall_targets
    }

In [43]:
class HybridModelWithAttention(nn.Module):
    """
    Hybrid LSTM model with attention mechanism for predicting per-task and overall scores.

    Args:
        input_size (int): Number of input features.
        embedding_sizes (dict): Dictionary with sizes of categorical embeddings.
        hidden_size (int): Number of hidden units in LSTM.
        num_layers (int): Number of LSTM layers.
        per_task_output_size (int): Output size for per-task predictions.
        overall_output_size (int): Output size for overall predictions.
        dropout_prob (float): Dropout probability.
    """
    def __init__(self, input_size, embedding_sizes, hidden_size, num_layers, per_task_output_size, overall_output_size, dropout_prob):
        super(HybridModelWithAttention, self).__init__()

        self.task_type_embedding = nn.Embedding(embedding_sizes['Task_Type'], embedding_dim=16)
        self.preferred_time_embedding = nn.Embedding(embedding_sizes['Preferred_Time'], embedding_dim=4)

        lstm_input_size = 16 + 4 + (input_size - 2)

        self.lstm = nn.LSTM(lstm_input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob, bidirectional=True)

        self.attention_layer = nn.Linear(hidden_size * 2, 1)

        self.per_task_fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(64, per_task_output_size)
        )

        self.overall_fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(64, overall_output_size)
        )

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, seq_len, input_size].

        Returns:
            per_task_outputs (torch.Tensor): Per-task predictions of shape [batch_size, seq_len, per_task_output_size].
            overall_output (torch.Tensor): Overall predictions of shape [batch_size, overall_output_size].
        """
        task_type = x[:, :, 0].long()
        preferred_time = x[:, :, 4].long()
        other_features = torch.cat([x[:, :, 1:4], x[:, :, 5:]], dim=2)

        task_type_embedded = self.task_type_embedding(task_type)
        preferred_time_embedded = self.preferred_time_embedding(preferred_time)

        lstm_input = torch.cat([task_type_embedded, preferred_time_embedded, other_features], dim=2)

        lstm_out, _ = self.lstm(lstm_input)

        per_task_outputs = self.per_task_fc(lstm_out)

        attn_weights = torch.softmax(self.attention_layer(lstm_out), dim=1)
        context_vector = torch.sum(attn_weights * lstm_out, dim=1)

        overall_output = self.overall_fc(context_vector)

        return per_task_outputs, overall_output

In [44]:
def initialize_model(feature_columns, df):
    """
    Initializes the model with the specified hyperparameters.

    Args:
        feature_columns (list): List of feature column names.
        df (pd.DataFrame): The preprocessed dataset.

    Returns:
        model (nn.Module): The initialized model.
    """
    embedding_sizes = {
        'Task_Type': df['Task_Type_Encoded'].nunique(),
        'Preferred_Time': df['Preferred_Time_Encoded'].nunique()
    }

    input_size = len(feature_columns)
    hidden_size = 64
    num_layers = 2
    per_task_output_size = 2
    overall_output_size = 2
    dropout_prob = 0.3

    model = HybridModelWithAttention(input_size, embedding_sizes, hidden_size, num_layers,
                                     per_task_output_size, overall_output_size, dropout_prob)
    model.to(device)
    return model

model = initialize_model(feature_columns, df)

In [45]:
def create_dataloaders(sequences, per_task_targets, overall_targets, batch_size=16):
    """
    Creates training and validation DataLoaders.

    Args:
        sequences (list): List of task sequences.
        per_task_targets (list): List of per-task target arrays.
        overall_targets (list): List of overall target arrays.
        batch_size (int): Batch size for DataLoaders.

    Returns:
        train_loader (DataLoader): DataLoader for training data.
        val_loader (DataLoader): DataLoader for validation data.
    """
    dataset = TaskSequenceDataset(sequences, per_task_targets, overall_targets)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, val_loader

train_loader, val_loader = create_dataloaders(sequences, per_task_targets, overall_targets)

In [46]:
def get_loss_functions_and_optimizer(model):
    """
    Defines loss functions and optimizer for training.

    Args:
        model (nn.Module): The model to be trained.

    Returns:
        criterion_per_task (nn.Module): Loss function for per-task predictions.
        criterion_overall (nn.Module): Loss function for overall predictions.
        optimizer (optim.Optimizer): Optimizer for model parameters.
    """
    criterion_per_task = nn.MSELoss()
    criterion_overall = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return criterion_per_task, criterion_overall, optimizer

criterion_per_task, criterion_overall, optimizer = get_loss_functions_and_optimizer(model)

In [47]:
def train_model(model, train_loader, val_loader, criterion_per_task, criterion_overall, optimizer, num_epochs=20):
    """
    Trains the model for the specified number of epochs.

    Args:
        model (nn.Module): The model to be trained.
        train_loader (DataLoader): DataLoader for training data.
        val_loader (DataLoader): DataLoader for validation data.
        criterion_per_task (nn.Module): Loss function for per-task predictions.
        criterion_overall (nn.Module): Loss function for overall predictions.
        optimizer (optim.Optimizer): Optimizer for model parameters.
        num_epochs (int): Number of epochs to train.

    Returns:
        None
    """
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch in train_loader:
            sequences = batch['sequence'].to(device)
            per_task_targets = batch['per_task_target'].to(device)
            overall_targets = batch['overall_target'].to(device)

            optimizer.zero_grad()

            per_task_outputs, overall_output = model(sequences)

            per_task_outputs_flat = per_task_outputs.view(-1, per_task_outputs.shape[-1])
            per_task_targets_flat = per_task_targets.view(-1, per_task_targets.shape[-1])

            loss_per_task = criterion_per_task(per_task_outputs_flat, per_task_targets_flat)
            loss_overall = criterion_overall(overall_output, overall_targets)

            loss = loss_per_task + loss_overall

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                sequences = batch['sequence'].to(device)
                per_task_targets = batch['per_task_target'].to(device)
                overall_targets = batch['overall_target'].to(device)

                per_task_outputs, overall_output = model(sequences)

                per_task_outputs_flat = per_task_outputs.view(-1, per_task_outputs.shape[-1])
                per_task_targets_flat = per_task_targets.view(-1, per_task_targets.shape[-1])

                loss_per_task = criterion_per_task(per_task_outputs_flat, per_task_targets_flat)
                loss_overall = criterion_overall(overall_output, overall_targets)

                loss = loss_per_task + loss_overall

                val_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')

train_model(model, train_loader, val_loader, criterion_per_task, criterion_overall, optimizer, num_epochs=100)

Epoch [1/100], Training Loss: 70.3729, Validation Loss: 42.5762
Epoch [2/100], Training Loss: 25.3706, Validation Loss: 15.0666
Epoch [3/100], Training Loss: 16.4723, Validation Loss: 13.9248
Epoch [4/100], Training Loss: 15.2739, Validation Loss: 11.9666
Epoch [5/100], Training Loss: 13.9387, Validation Loss: 11.3143
Epoch [6/100], Training Loss: 13.6568, Validation Loss: 10.8003
Epoch [7/100], Training Loss: 12.9649, Validation Loss: 10.5338
Epoch [8/100], Training Loss: 12.8187, Validation Loss: 10.3457
Epoch [9/100], Training Loss: 12.3011, Validation Loss: 10.2823
Epoch [10/100], Training Loss: 12.3983, Validation Loss: 10.1727
Epoch [11/100], Training Loss: 12.1433, Validation Loss: 10.2344
Epoch [12/100], Training Loss: 12.5635, Validation Loss: 10.1404
Epoch [13/100], Training Loss: 12.0089, Validation Loss: 10.2526
Epoch [14/100], Training Loss: 12.0810, Validation Loss: 10.1308
Epoch [15/100], Training Loss: 11.9058, Validation Loss: 10.1362
Epoch [16/100], Training Loss: 11.

In [48]:
def save_model(model, file_path):
    """
    Saves the trained model to the specified file.

    Args:
        model (nn.Module): The trained model.
        file_path (str): Path to save the model.

    Returns:
        None
    """
    torch.save(model.state_dict(), file_path)

save_model(model, 'hybrid_model_with_attention.pth')